# <center> Pandas, part 2 </center>
### By the end of this talk, you will be able to
   - modify/clean columns
   - evaluate the runtime of your scripts
   - merge and append data frames

###  <font color='LIGHTGRAY'>By the end of this talk, you will be able to</font>
   - **modify/clean columns**
   - **evaluate the runtime of your scripts**
   -  <font color='LIGHTGRAY'>merge and append data frames</font>

## What is data cleaning?
- the data you scrape from sites are often not in a format you can directly work with
    - the temp column in weather.csv contains the temperature value but also other strings
    - the year column in imdb.csv contains the year the movie came out but also - or () and roman numbers
- data cleaning means that you bring the data in a format that's easier to analyze/visualize 

## How is it done?
- you read in your data into a pandas data frame and either 
    - modify the values of a column
    - create a new column that contains the modified values
- either works, I'll show you how to do both
- ADVICE: when you save the modified data frame, it is usually good practice to not overwrite the original csv or excel file that you scraped. 
    - save the modified data into a new file instead

## Ways to modify a column
- there are several ways to do this
    - with a for loop
    - with a list comprehension
    - using the .apply method
- we will compare run times
    - investigate which approach is faster
    - important when you work with large datasets (>1,000,000 lines)

## The task: runtime column in imdb.csv
- the column is string in the format 'n min' where n is the length of the movie in minutes
- for plotting purposes, it is better if the runtime is not a string but a number (float or int)
    - you can't create a histogram of runtime using strings
- task: clean the runtime column and convert it to float

## Approach 1: for loop

In [1]:
# read in the data
import pandas as pd
df_imdb = pd.read_csv('data/imdb.csv')
print(df_imdb.head())

                            title        year  \
0  Get Paid, Get Laid & Get Lucky      (2019)   
1           Izzy Got the Frizzies      (2019)   
2                          Blakus      (2019)   
3                             DNA  (I) (2019)   
4               Matar a un Muerto      (2019)   

                                        genre  runtime  rating  \
0         \nCrime, Drama, Romance               86 min     7.0   
1  \nAnimation, Action, Adventure                  NaN     4.8   
2        \nComedy, Drama, Romance               80 min     7.1   
3                         \nDrama              126 min     7.8   
4                      \nThriller               87 min     7.6   

                                            synopsis         director  vote  
0  \n    A young man,driven only by his dreams,wi...    Goran Pavisic    65  
1  \n    Whether you have naturally frizzy hair o...       Ki-duk Kim    11  
2  \n    Louise and Kaspars meet on a trip to cel...     Alise Zarina   31

In [2]:
import time

start = time.time() # start the clock

for i in range(100): # repeat everything 100 times to get better estimate of elapsed time
    # the actual code to clean the runtime column comes here
    runtime_lst = []
    for x in df_imdb['runtime']:
        if type(x) == str:
            runtime = float(x[:-4].replace(',',''))
        else:
            runtime = 0e0
        runtime_lst.append(runtime)
    df_imdb['runtime min'] = runtime_lst
    
end = time.time() # stop the timer

print('cpu time = ',end-start,'sec')

cpu time =  0.41921210289001465 sec


## Approach 2: list comprehension

In [3]:
start = time.time() # start the clock

for i in range(100): # repeat everything 100 times to get better estimate of elapsed time
    # the actual code to clean the runtime column comes here
    df_imdb['runtime min'] = [float(x[:-4].replace(',','')) if type(x) == str else 0e0 for x in df_imdb['runtime']]
end = time.time() # stop the timer

print('cpu time = ',end-start,'sec')

cpu time =  0.31636476516723633 sec


## Approach 3: the .apply method

In [4]:
def clean_runtime(x):
    if type(x) == str:
        runtime = float(x[:-4].replace(',',''))
    else:
        runtime = 0e0
    return  runtime

start = time.time() # start the clock

for i in range(100): # repeat everything 100 times to get better estimate of elapsed time
    # the actual code to clean the runtime column comes here
    df_imdb['runtime min'] = df_imdb['runtime'].apply(clean_runtime)
end = time.time() # stop the timer

print('cpu time = ',end-start,'sec')

cpu time =  0.30475878715515137 sec


## Summary
- the for loop is slower
- the list comprehension and the apply method are equally quick
- it is down to personal preference to choose between list comprehension and .apply
- **the same ranking is not quaranteed for a different task!**
    - **always try a few different approaches if runtime is an issue (you work with large data)!**

## Exercise 1
Clean the `temp` column in the `data/weather.csv` file. The new temperature column should be an integer or a float. Work through at least one of the approaches we discussed. If you have time, work through all three methods. If you have even more time, look for other approaches to clean a column and time it using the `runtime` column of the imdb.csv. Try to beat my cpu time and find an even faster approach! :)

###  <font color='LIGHTGRAY'>By the end of this talk, you will be able to</font>
   - <font color='LIGHTGRAY'>modify/clean columns</font>
   - <font color='LIGHTGRAY'>evaluate the runtime of your scripts</font>
   -  **merge and append data frames**

### How to merge dataframes?

Merge - data are distributed in multiple files

In [5]:
# We have two datasets from two hospitals

hospital1 = {'ID':['ID1','ID2','ID3','ID4','ID5','ID6','ID7'],'col1':[5,8,2,6,0,2,5],'col2':['y','j','w','b','a','b','t']}
df1 = pd.DataFrame(data=hospital1)
print(df1)

hospital2 = {'ID':['ID2','ID5','ID6','ID10','ID11'],'col3':[12,76,34,98,65],'col2':['q','u','e','l','p']}
df2 = pd.DataFrame(data=hospital2)
print(df2)



    ID  col1 col2
0  ID1     5    y
1  ID2     8    j
2  ID3     2    w
3  ID4     6    b
4  ID5     0    a
5  ID6     2    b
6  ID7     5    t
     ID  col3 col2
0   ID2    12    q
1   ID5    76    u
2   ID6    34    e
3  ID10    98    l
4  ID11    65    p


In [6]:
# we are interested in only patients from hospital1
#df_left = df1.merge(df2,how='left',on='ID') # IDs from the left dataframe (df1) are kept
#print(df_left)

# we are interested in only patients from hospital2
#df_right = df1.merge(df2,how='right',on='ID') # IDs from the right dataframe (df2) are kept
#print(df_right)

# we are interested in patiens who were in both hospitals
#df_inner = df1.merge(df2,how='inner',on='ID') # merging on IDs present in both dataframes
#print(df_inner)

# we are interested in all patients who visited at least one of the hospitals
#df_outer = df1.merge(df2,how='outer',on='ID')  # merging on IDs present in any dataframe
#print(df_outer)

### How to append dataframes?

Append - new data comes in over a period of time. E.g., one file per month/quarter/fiscal year etc.


You want to combine these files into one data frame.

In [7]:
#df_append = df1.append(df2) # note that rows with ID2, ID5, and ID6  are duplicated! Indices are duplicated too.
#print(df_append)

df_append = df1.append(df2,ignore_index=True) # note that rows with ID2, ID5, and ID6  are duplicated! 
#print(df_append)

d3 = {'ID':['ID23','ID94','ID56','ID17'],'col1':['rt','h','st','ne'],'col2':[23,86,23,78]}
df3 = pd.DataFrame(data=d3)
#print(df3)

df_append = df1.append([df2,df3],ignore_index=True) # multiple dataframes can be appended to df1
print(df_append)



      ID col1 col2  col3
0    ID1    5    y   NaN
1    ID2    8    j   NaN
2    ID3    2    w   NaN
3    ID4    6    b   NaN
4    ID5    0    a   NaN
5    ID6    2    b   NaN
6    ID7    5    t   NaN
7    ID2  NaN    q  12.0
8    ID5  NaN    u  76.0
9    ID6  NaN    e  34.0
10  ID10  NaN    l  98.0
11  ID11  NaN    p  65.0
12  ID23   rt   23   NaN
13  ID94    h   86   NaN
14  ID56   st   23   NaN
15  ID17   ne   78   NaN


### Exercise 2
- Create three data frames from raw_data_1, 2, and 3.
- Append the first two data frames and assign it to df_append.
- Merge the third data frame with df_append such that only subject_ids from df_append are present. 
- Assign the new data frame to df_merge. 
- How many rows and columns do we have in df_merge?


In [8]:

raw_data_1 = {
        'subject_id': ['1', '2', '3', '4', '5'],
        'first_name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'], 
        'last_name': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches']}

raw_data_2 = {
        'subject_id': ['6', '7', '8', '9', '10'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'], 
        'last_name': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']}

raw_data_3 = {
        'subject_id': ['1', '2', '3', '4', '5', '7', '8', '9', '10', '11'],
        'test_id': [51, 15, 15, 61, 16, 14, 15, 1, 61, 16]}


### Always check that the resulting dataframe is what you wanted to end up with!
- small toy datasets are ideal to test your code.

### If you need to do a more complicated dataframe operation, check out pd.concat()!
